In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import functions as func

In [ ]:
class Logistic(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Logistic, self).__init__()
        
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

## Dataset

In [ ]:
X = np.random.uniform(0, 1, size=(500,))

Y = func.pillar(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

plt.scatter(X, Y)
plt.title('Data')
plt.show()

train_ds = data.TensorDataset(
    torch.tensor(X_train, dtype=torch.float).unsqueeze(-1),
    torch.tensor(Y_train, dtype=torch.float)
)
test_ds = data.TensorDataset(
    torch.tensor(X_test, dtype=torch.float).unsqueeze(-1),
    torch.tensor(Y_test, dtype=torch.float)
)

## Training

In [ ]:
LEARNING_RATE = 1e-1
EPOCHS = 1000
BATCH_SIZE = 16
HIDDEN_LAYER_SIZE = 2

train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE)
test_loader = data.DataLoader(test_ds)

l1 = Logistic(1, HIDDEN_LAYER_SIZE)
l2 = Logistic(HIDDEN_LAYER_SIZE, 1)

model = nn.Sequential(l1, l2)

cost = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loss = []

model.train()

for epoch in tqdm(range(EPOCHS), 'Epoch'):
    epoch_loss = []

    for x, y in train_loader:
        # Compute prediction and loss
        pred = model(x)
        loss = cost(pred, y.unsqueeze(-1))

        # Backpropagation
        opt.zero_grad()
        loss.backward()
        opt.step() 
    
        epoch_loss.append(loss.detach())

    train_loss.append(torch.tensor(epoch_loss).mean())

In [ ]:
model.eval()

x, y = test_ds[:]

output = model(x).detach().squeeze()

loss = cost(output, y)
print('test loss = ', loss)

In [ ]:
plt.plot(train_loss)
plt.title('Training loss')
plt.show()

## Neuron visualization

In [ ]:
x_lin = torch.linspace(0, 1, 100).unsqueeze(-1)
f = model(x_lin).detach().numpy()

plt.scatter(X, Y, alpha=0.4)
plt.plot(x_lin, f)
plt.title('Trained model')
plt.show()

In [ ]:
def plot_l1_neurons():
    f = l1(x_lin).detach()

    plt.scatter(X, Y, alpha=0.4)

    for neuron in range(HIDDEN_LAYER_SIZE):
        plt.plot(x_lin, f[:, neuron].numpy(), label=f'neuron {neuron}')

    plt.legend()
    plt.title('Hidden layer neurons')
    plt.show()

plot_l1_neurons()

In [ ]:
if HIDDEN_LAYER_SIZE == 2:
    f = l1(x_lin).detach()

    f_n0 = f[:, 0]
    f_n1 = f[:, 1]

    x_lin2 = torch.cartesian_prod(f_n0, f_n1)
    f = l2(x_lin2).detach().numpy().reshape((len(f_n0), len(f_n1)))

    plt.subplot(1, 2, 1)
    plt.title('Output layer')
    plt.imshow(f, extent=[0, 1, 0, 1])

    plt.subplot(1, 2, 2)
    plot_l1_neurons()